## Imports

In [2]:
import pandas as pd
import numpy as np
import json
import nltk
import streamlit as st
import spacy
import pyLDAvis
import pyLDAvis.sklearn
import pyLDAvis.gensim

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction import text 
from sklearn.decomposition import LatentDirichletAllocation

import gensim
pyLDAvis.enable_notebook()

# local library
from preproc import *

/home/kfinity/anaconda3/envs/python38/lib/python3.8/site-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [3]:
with open('speeches.json') as f:
    speeches = json.load(f)

/home/kfinity/anaconda3/envs/python38/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
bow = create_bow(speeches)

/home/kfinity/anaconda3/envs/python38/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
2020-11-09 20:25:05.636 INFO    numexpr.utils: Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2020-11-09 20:25:05.637 INFO    numexpr.utils: NumExpr defaulting to 8 threads.


## Preprocess

In [5]:
TOKEN = tokenize(bow)

/home/kfinity/anaconda3/envs/python38/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
VOCAB = TOKEN.term_str.value_counts().to_frame().rename(columns={'index':'term_str', 'term_str':'n'})\
    .sort_index().reset_index().rename(columns={'index':'term_str'})
VOCAB.index.name = 'term_id'

/home/kfinity/anaconda3/envs/python38/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [7]:
# add term_id back to token
TOKEN['term_id'] = TOKEN.term_str.map(VOCAB.reset_index().set_index('term_str').term_id)

/home/kfinity/anaconda3/envs/python38/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [8]:
NOUNS = TOKEN[TOKEN.pos.str.match(r'^NNS?$')]\
    .groupby(['id','speaker']).term_str\
    .apply(lambda x: ' '.join(x))\
    .to_frame()\
    .rename(columns={'term_str':'speech'})

/home/kfinity/anaconda3/envs/python38/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [9]:
NOUNS

/home/kfinity/anaconda3/envs/python38/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,,speech
id,speaker,
-8KiohxTJ0Y,biden,wife sister folks things clothes count campaig...
-bHGb17kejM,trump,thank warning warning requests sir updates fra...
-oLFDfEAXa0,trump,county news conference service election day ye...
0pJnrHKoBHY,trump,music election tomorrow weve state cars gonna ...
17Xl75GwOaM,trump,thank vice president place village confidence ...
...,...,...
zIVdxsiu_r4,biden,reminder todays event distancing guideline gui...
zYz-QRYtR9U,trump,secretary carson secretary devos dr larry yarn...
zaaTZkqsaxY,trump,nothing thing closing rating everythin...


In [10]:
stop_words = text.ENGLISH_STOP_WORDS.union(['music','applause'])

/home/kfinity/anaconda3/envs/python38/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [11]:
biden = NOUNS.query('speaker == "biden"').speech
trump = NOUNS.query('speaker == "trump"').speech
pence = NOUNS.query('speaker == "pence"').speech
harris = NOUNS.query('speaker == "harris"').speech

/home/kfinity/anaconda3/envs/python38/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Scikit-Learn

In [12]:
tf_vectorizer_biden = CountVectorizer(strip_accents = 'unicode',
                                stop_words = stop_words,
                                lowercase = True,
                                token_pattern = r'\b[a-zA-Z]{3,}\b',
                                max_df = .75, 
                                min_df = .1)
tf_vectorizer_trump = CountVectorizer(strip_accents = 'unicode',
                                stop_words = stop_words,
                                lowercase = True,
                                token_pattern = r'\b[a-zA-Z]{3,}\b',
                                max_df = .75, 
                                min_df = .1)
tf_vectorizer_pence = CountVectorizer(strip_accents = 'unicode',
                                stop_words = stop_words,
                                lowercase = True,
                                token_pattern = r'\b[a-zA-Z]{3,}\b',
                                max_df = .75, 
                                min_df = .1)
tf_vectorizer_harris = CountVectorizer(strip_accents = 'unicode',
                                stop_words = stop_words,
                                lowercase = True,
                                token_pattern = r'\b[a-zA-Z]{3,}\b',
                                max_df = .75, 
                                min_df = .1)
dtm_tf_biden = tf_vectorizer_biden.fit_transform(biden)
dtm_tf_trump = tf_vectorizer_trump.fit_transform(trump)
dtm_tf_pence = tf_vectorizer_pence.fit_transform(pence)
dtm_tf_harris = tf_vectorizer_harris.fit_transform(harris)

/home/kfinity/anaconda3/envs/python38/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [13]:
# for TF DTM
lda_tf_biden = LatentDirichletAllocation(n_components=8, random_state=0)
lda_tf_biden.fit(dtm_tf_biden)
lda_tf_trump = LatentDirichletAllocation(n_components=8, random_state=0)
lda_tf_trump.fit(dtm_tf_trump)
lda_tf_pence = LatentDirichletAllocation(n_components=8, random_state=0)
lda_tf_pence.fit(dtm_tf_pence)
lda_tf_harris = LatentDirichletAllocation(n_components=8, random_state=0)
lda_tf_harris.fit(dtm_tf_harris)

/home/kfinity/anaconda3/envs/python38/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


LatentDirichletAllocation(n_components=8, random_state=0)

In [14]:
pyLDAvis.sklearn.prepare(lda_tf_biden, dtm_tf_biden, tf_vectorizer_biden)

/home/kfinity/anaconda3/envs/python38/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7     -0.016090  0.081336       1        1  34.031071
1     -0.007392 -0.009955       2        1  29.201566
5     -0.014371 -0.004093       3        1  12.146068
0     -0.216372 -0.162980       4        1   5.962562
4      0.145954 -0.068087       5        1   5.607919
2      0.043120 -0.003574       6        1   5.290650
3     -0.077904  0.216949       7        1   5.225182
6      0.143055 -0.049597       8        1   2.534982, topic_info=           Term        Freq       Total Category  logprob  loglift
448         joe  121.000000  121.000000  Default  30.0000  30.0000
898       trump  231.000000  231.000000  Default  29.0000  29.0000
915     vaccine   53.000000   53.000000  Default  28.0000  28.0000
493         lot  196.000000  196.000000  Default  27.0000  27.0000
300        fact  170.000000  170.000000  Default  26.0000  26.0000
..          ...         ...         ...      ...      ...      ...
867       thank    4.447990   83.485574   Topic8  -4.8901   0.7428
356  government    4.631509  103.198743   Topic8  -4.8496   0.5712
300        fact    4.939427  170.160932   Topic8  -4.7853   0.1355
493         lot    4.731755  196.024019   Topic8  -4.8282  -0.0490
922        vice    3.723838   93.085218   Topic8  -5.0678   0.4562

[509 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
2         1  0.566245  access
2         2  0.022650  access
2         3  0.090599  access
2         5  0.294447  access
2         8  0.022650  access
...     ...       ...     ...
975       2  0.199295   youre
975       3  0.117233   youre
975       6  0.058616   youre
975       7  0.128956   youre
975       8  0.023447   youre

[1459 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 2, 6, 1, 5, 3, 4, 7])

In [15]:
pyLDAvis.sklearn.prepare(lda_tf_trump, dtm_tf_trump, tf_vectorizer_trump)

/home/kfinity/anaconda3/envs/python38/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.019807  0.001557       1        1  41.725593
0      0.036565  0.080624       2        1  19.585766
5     -0.004743  0.108261       3        1  15.324495
6     -0.016662  0.085364       4        1  11.428419
2      0.198083 -0.010246       5        1   5.551546
7     -0.022793 -0.056874       6        1   3.522701
1     -0.217855 -0.036233       7        1   1.636728
3      0.047212 -0.172453       8        1   1.224751, topic_info=         Term        Freq       Total Category  logprob  loglift
470     gonna  662.000000  662.000000  Default  30.0000  30.0000
842    police  254.000000  254.000000  Default  29.0000  29.0000
83    ballots  225.000000  225.000000  Default  28.0000  28.0000
104     biden  509.000000  509.000000  Default  27.0000  27.0000
37    america  351.000000  351.000000  Default  26.0000  26.0000
...       ...         ...         ...      ...      ...      ...
255     crime    5.071189  149.552811   Topic8  -4.9763   1.0184
244    couple    5.071196  163.765799   Topic8  -4.9763   0.9276
777     order    5.071194  192.065075   Topic8  -4.9763   0.7682
1274    women    5.071190  257.325235   Topic8  -4.9763   0.4757
470     gonna    5.071179  662.855138   Topic8  -4.9763  -0.4706

[521 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
1         1  0.820603  abortion
1         2  0.018650  abortion
1         4  0.139876  abortion
1         6  0.009325  abortion
10        1  0.440790   actions
...     ...       ...       ...
1287      7  0.007664      york
1287      8  0.022991      york
1288      2  0.119550     yorks
1288      3  0.657528     yorks
1288      4  0.179326     yorks

[1728 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 1, 6, 7, 3, 8, 2, 4])

In [16]:
pyLDAvis.sklearn.prepare(lda_tf_harris, dtm_tf_harris, tf_vectorizer_harris)

/home/kfinity/anaconda3/envs/python38/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.022935  0.027536       1        1  37.261205
6      0.036563 -0.013739       2        1  31.069244
1     -0.032978  0.122147       3        1  13.111810
5     -0.226781 -0.110531       4        1   9.099959
2      0.154606 -0.154089       5        1   8.064027
4      0.025275  0.089567       6        1   1.190824
3      0.011765  0.017511       7        1   0.181905
7      0.008614  0.021597       8        1   0.021027, topic_info=           Term       Freq       Total Category  logprob  loglift
68        biden  100.00000  100.000000  Default  30.0000  30.0000
113        care   61.00000   61.000000  Default  29.0000  29.0000
956        year   35.00000   35.000000  Default  28.0000  28.0000
344     georgia   35.00000   35.000000  Default  27.0000  27.0000
793       state   52.00000   52.000000  Default  26.0000  26.0000
..          ...        ...         ...      ...      ...      ...
598   pandemics    0.00177    2.496932   Topic8  -6.8721   1.2151
292      expand    0.00177    2.496932   Topic8  -6.8721   1.2151
48   background    0.00177    2.483352   Topic8  -6.8721   1.2206
308      father    0.00177    6.015927   Topic8  -6.8721   0.3358
943      womans    0.00177    2.392780   Topic8  -6.8721   1.2577

[510 rows x 6 columns], token_table=      Topic      Freq            Term
term                                 
1         1  0.410358        abortion
1         5  0.410358        abortion
6         2  0.925036  accountability
7         1  0.868780             act
7         2  0.066829             act
...     ...       ...             ...
959       2  0.982927            york
962       1  0.233502           youre
962       2  0.525379           youre
962       3  0.087563           youre
962       5  0.145939           youre

[812 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 7, 2, 6, 3, 5, 4, 8])

In [17]:
pyLDAvis.sklearn.prepare(lda_tf_pence, dtm_tf_pence, tf_vectorizer_pence)

/home/kfinity/anaconda3/envs/python38/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.055345  0.062044       1        1  65.429812
6      0.056220  0.156537       2        1  11.904901
0      0.123383 -0.142804       3        1   8.758409
4      0.093649 -0.014724       4        1   7.549370
1     -0.182951 -0.032083       5        1   5.676879
7     -0.108430 -0.003953       6        1   0.630494
5     -0.019000 -0.012849       7        1   0.026792
2     -0.018216 -0.012168       8        1   0.023343, topic_info=            Term        Freq       Total Category  logprob  loglift
711        space   61.000000   61.000000  Default  30.0000  30.0000
398          joe  279.000000  279.000000  Default  29.0000  29.0000
795        trump  292.000000  292.000000  Default  28.0000  28.0000
156  congressman   43.000000   43.000000  Default  27.0000  27.0000
602     promises   48.000000   48.000000  Default  26.0000  26.0000
..           ...         ...         ...      ...      ...      ...
81      campaign    0.003907   21.976534   Topic8  -6.7639  -0.2722
820        voice    0.003907   17.570420   Topic8  -6.7639  -0.0485
238      embassy    0.003907    8.960402   Topic8  -6.7639   0.6249
117      choices    0.003907    3.480198   Topic8  -6.7639   1.5707
316       ground    0.003907   10.484868   Topic8  -6.7639   0.4678

[495 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
0         4  0.658919     ability
0         5  0.329460     ability
1         1  0.479457    abortion
1         2  0.405694    abortion
1         4  0.018441    abortion
...     ...       ...         ...
855       5  0.692512  worldclass
855       6  0.346256  worldclass
858       1  0.333731        yeah
858       3  0.667461        yeah
861       3  0.937169        york

[965 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 7, 1, 5, 2, 8, 6, 3])

In [18]:
biden_viz = pyLDAvis.sklearn.prepare(lda_tf_biden, dtm_tf_biden, tf_vectorizer_biden)
pyLDAvis.save_html(biden_viz, 'biden_viz.html')

/home/kfinity/anaconda3/envs/python38/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [19]:
trump_viz = pyLDAvis.sklearn.prepare(lda_tf_trump, dtm_tf_trump, tf_vectorizer_trump)
pyLDAvis.save_html(trump_viz, 'trump_viz.html')

/home/kfinity/anaconda3/envs/python38/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [20]:
harris_viz = pyLDAvis.sklearn.prepare(lda_tf_harris, dtm_tf_harris, tf_vectorizer_harris)
pyLDAvis.save_html(harris_viz, 'harris_viz.html')

/home/kfinity/anaconda3/envs/python38/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [21]:
pence_viz = pyLDAvis.sklearn.prepare(lda_tf_pence, dtm_tf_pence, tf_vectorizer_pence)
pyLDAvis.save_html(pence_viz, 'pence_viz.html')

/home/kfinity/anaconda3/envs/python38/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
